In [20]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
def load_housing_data():
    return pd.read_csv("dataset/housing.csv")

In [21]:
housing=load_housing_data()


In [22]:
housing=housing.reset_index()

In [23]:
housing["income_cat"]=np.ceil(housing["median_income"]/1.5)
housing["income_cat"].where(housing["income_cat"]<5,5.0,inplace=True)


In [24]:
from sklearn.model_selection import StratifiedShuffleSplit
split=StratifiedShuffleSplit(n_splits=3,test_size=0.2,random_state=42)
for train_index,test_index in split.split(housing,housing["income_cat"]):
        strat_train_set=housing.loc[train_index]
        strat_test_set=housing.loc[test_index]


In [25]:
for set in strat_train_set,strat_test_set :
    set.drop(columns=['income_cat'])

In [26]:
housing=strat_train_set.drop("median_house_value",axis=1)
housing_labels=strat_train_set["median_house_value"].copy()
housing_num=housing.drop("ocean_proximity",axis=1)

In [27]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")
housing_num=housing.drop("ocean_proximity",axis=1)
imputer.fit(housing_num)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [28]:
X=imputer.transform(housing_num)


In [29]:
housing_tr=pd.DataFrame(X,columns=housing_num.columns)


In [30]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
housing_cat=housing["ocean_proximity"]
housing_cat_encoded=encoder.fit_transform(housing_cat)

In [31]:
housing_tr_1=pd.DataFrame(housing_cat_encoded,columns=["ocean_proximity"])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_tr)
housing_tr=scaler.transform(housing_tr)
housing_tr=pd.DataFrame(housing_tr,columns=housing_num.columns)


In [32]:
housing_tr=housing_tr.join(housing_tr_1,lsuffix='_housing_tr', rsuffix='_housing_tr_1')
size1=housing_tr['latitude'].size
ones_matrix=np.ones(size1)
ones_matrix_1=ones_matrix.reshape((size1,1))
print(housing_tr.head())


      index  longitude  latitude  housing_median_age  total_rooms  \
0 -0.527959   0.684384 -0.738029            1.464498    -0.626342   
1  0.739448   1.258295 -1.323678           -0.288400    -0.059509   
2 -0.098373  -1.087254  0.545714           -1.005495     0.556133   
3  0.160388   0.779204 -0.859844           -0.846141     0.481538   
4  0.071108   0.824119 -0.948863            0.667726    -0.251063   

   total_bedrooms  population  households  median_income  income_cat  \
0       -0.650005   -0.204092   -0.625833      -0.753784   -0.954456   
1       -0.456875   -0.314209   -0.390613       0.985693    0.942051   
2        0.198812    0.325518    0.291527       0.269602   -0.006202   
3        0.375251    0.343871    0.398683      -0.113819   -0.006202   
4       -0.063463   -0.658543   -0.286070       0.650443    0.942051   

   ocean_proximity  
0                0  
1                0  
2                0  
3                0  
4                0  


In [33]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
lassoreg = Lasso(alpha=2,normalize=True, max_iter=1e5)
lassoreg.fit(housing_tr,housing_labels)
y_pred = lassoreg.predict(housing_tr)
lin_mse=mean_squared_error(housing_labels,y_pred)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

69424.92265989268
